In [20]:
pip install gradio scikit-image scikit-learn cupy matplotlib


  Using cached cupy-13.3.0.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for cupy
Failed to build cupy
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [59 lines of output]
      Generating cache key from header files...
      Cache key (1610 files matching C:\Users\Marwan Shamsan\AppData\Local\Temp\pip-install-a444wtr1\cupy_fd9d91c0a655455596ca8a90198f47dd\cupy\_core\include\**): 8aad1231f702316b675b9ad296f56a95316298c2
      Clearing directory: C:\Users\Marwan Shamsan\AppData\Local\Temp\pip-install-a444wtr1\cupy_fd9d91c0a655455596ca8a90198f47dd\cupy\.data
      Looking for NVTX: C:\Program Files\NVIDIA Corporation\Nsight Systems *\target-windows-x64\nvtx
      Using NVTX at: C:\Program Files\NVIDIA Corporation\Nsight Systems 2024.6.2\target-windows-x64\nvtx
      
      -------- Configuring Module: cuda --------
      Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      ******************************************

In [21]:
import gradio as gr
import pickle
import numpy as np
import cupy as cp
import cv2
from skimage import morphology, filters, segmentation, exposure, feature, transform
from scipy.ndimage import distance_transform_edt, label as nd_label
from sklearn.preprocessing import MinMaxScaler
from skimage.color import rgb2gray
from joblib import load  # Import joblib

# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {}
for model_name, model_path in models.items():
    loaded_models[model_name] = load(model_path)  # Use joblib.load()


# Step 1: Anisotropic Diffusion Filter
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = cp.asarray(image.copy())  
    for _ in range(iterations):
        gradient_magnitude = cp.gradient(image)
        c = 1.0 / (1.0 + (gradient_magnitude[0] / beta) ** 2)
        diff_north = cp.roll(image, shift=1, axis=0) - image
        diff_south = cp.roll(image, shift=-1, axis=0) - image
        diff_east = cp.roll(image, shift=1, axis=1) - image
        diff_west = cp.roll(image, shift=-1, axis=1) - image
        image += alpha * (c * (diff_north + diff_south + diff_east + diff_west))
    return cp.asnumpy(image)  

# Step 2: Skull Stripping
def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=1500):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

# Step 3: Top-Hat Filtering
def top_hat_filtering(image, kernel_size=30):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    return cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)

# Step 4: Contrast Enhancement
def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    return ((contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image)) * 255).astype(np.uint8)

# Step 5: Binary Segmentation
def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    return image > thresh, thresh

# Step 6: Watershed Segmentation
def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    image = cp.asarray(image)
    distance = cp.asarray(distance_transform_edt(cp.asnumpy(image)))
    local_maxi = feature.peak_local_max(cp.asnumpy(distance), labels=cp.asnumpy(image), footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = cp.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(np.array(local_maxi).T)] = True
    markers = nd_label(cp.asnumpy(local_maxi_mask))[0]
    segmented_image = segmentation.watershed(-cp.asnumpy(distance), markers, mask=cp.asnumpy(image))
    return cp.asnumpy(segmented_image), np.max(segmented_image)

# Step 7: Morphological Operations
def apply_morphological_operations(image, kernel_size=5):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    image = image.astype(np.uint8)
    closed = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)
    return closed, opened

# Full preprocessing pipeline
def preprocess_pipeline(image):
    # Convert to grayscale
    grayscale_image = rgb2gray(image)

    # Apply the full pipeline
    adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
    skull_stripped_image = skull_stripping(adf_filtered_image)
    tophat_image = top_hat_filtering(skull_stripped_image)
    contrasted_image = contrast_enhancement(tophat_image)
    binary_image, thresh = get_binary(contrasted_image, thresh=230)
    segmented_image, nbr_labels = watershed_segmentation(binary_image, morphology.disk(10))
    closed_image_post, opened_image = apply_morphological_operations(segmented_image)

    # Resize to match model input (256x256)
    resized_image = transform.resize(binary_image, (256, 256))

    # Flatten the image for model input
    flattened_image = resized_image.flatten().reshape(1, -1)

    # Scale features
    scaler = MinMaxScaler()
    scaled_features = scaler.fit_transform(flattened_image)

    return scaled_features, resized_image, contrasted_image

# Classification Function
def classify_image(image, model_name):
    # Preprocess image
    scaled_features, preprocessed_image, contrast_image = preprocess_pipeline(image)

    # Get selected model
    model = loaded_models[model_name]

    # Make prediction
    prediction = model.predict(scaled_features)

    return preprocessed_image, f"Predicted Label: {prediction[0]}"

# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(
                    choices=list(models.keys()), label="Select Model", value="Random Forest"
                )
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Preprocessing
        preprocess_button.click(
            lambda image: preprocess_pipeline(image)[1], inputs=image_input, outputs=preprocess_output
        )

        # Classification
        classify_button.click(
            classify_image,
            inputs=[image_input, model_dropdown],
            outputs=[preprocess_output, prediction_output],
        )

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


In [22]:
import gradio as gr
import pickle
import numpy as np
import cupy as cp
import cv2
from skimage import morphology, filters, segmentation, exposure, feature, transform
from scipy.ndimage import distance_transform_edt, label as nd_label
from sklearn.preprocessing import MinMaxScaler
from skimage.color import rgb2gray
from joblib import load

# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {}
for model_name, model_path in models.items():
    loaded_models[model_name] = load(model_path)

def preprocess_pipeline(image):
    try:
        # Convert to grayscale
        grayscale_image = rgb2gray(image)

        # Apply preprocessing steps
        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        skull_stripped_image = skull_stripping(adf_filtered_image)
        tophat_image = top_hat_filtering(skull_stripped_image)
        contrasted_image = contrast_enhancement(tophat_image)
        binary_image, _ = get_binary(contrasted_image, thresh=230)
        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        closed_image_post, _ = apply_morphological_operations(segmented_image)

        # Resize for model input
        resized_image = transform.resize(binary_image, (256, 256))
        resized_image = (resized_image * 255).astype(np.uint8)  # Ensure uint8 format

        # Flatten and scale the image
        flattened_image = resized_image.flatten().reshape(1, -1)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_image, contrasted_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise


def classify_image(image, model_name):
    try:
        scaled_features, preprocessed_image, contrast_image = preprocess_pipeline(image)

        # Ensure preprocessed_image is in uint8 for Gradio
        preprocessed_image = (preprocessed_image * 255).astype(np.uint8)

        # Predict using the selected model
        model = loaded_models[model_name]
        prediction = model.predict(scaled_features)

        return preprocessed_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"


# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(
                    choices=list(models.keys()), label="Select Model", value="Random Forest"
                )
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Preprocessing
        preprocess_button.click(
            lambda image: preprocess_pipeline(image)[1], inputs=image_input, outputs=preprocess_output
        )

        # Classification
        classify_button.click(
            classify_image,
            inputs=[image_input, model_dropdown],
            outputs=[preprocess_output, prediction_output],
        )

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


In [23]:
import gradio as gr
import numpy as np
import cupy as cp
import cv2
from skimage import morphology, filters, segmentation, exposure, transform, feature  # 🔥 Added `feature`
from skimage.color import rgb2gray
from scipy.ndimage import distance_transform_edt, label as nd_label
from sklearn.preprocessing import MinMaxScaler
from joblib import load


# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {}
for model_name, model_path in models.items():
    loaded_models[model_name] = load(model_path)

# Step 1: Anisotropic Diffusion Filter

# Step 1: Anisotropic Diffusion Filter (ADF)
# Step 1: Anisotropic Diffusion Filter
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = cp.asarray(image.copy())  # Move image to GPU
    for _ in range(iterations):
        gradient_magnitude = cp.gradient(image)
        c = 1.0 / (1.0 + (gradient_magnitude[0] / beta) ** 2)
        diff_north = cp.roll(image, shift=1, axis=0) - image
        diff_south = cp.roll(image, shift=-1, axis=0) - image
        diff_east = cp.roll(image, shift=1, axis=1) - image
        diff_west = cp.roll(image, shift=-1, axis=1) - image
        image += alpha * (c * (diff_north + diff_south + diff_east + diff_west))
    return cp.asnumpy(image)  # Move back to CPU

# Step 2: Skull Stripping
def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=1500):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

# Step 3: Top-Hat Filtering
def top_hat_filtering(image, kernel_size=30):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    return cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)

# Step 4: Contrast Enhancement
def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    return ((contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image)) * 255).astype(np.uint8)

# Step 5: Binary Segmentation
def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    return image > thresh, thresh

# Step 6: Watershed Segmentation
def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    image = cp.asarray(image)
    distance = cp.asarray(distance_transform_edt(cp.asnumpy(image)))
    local_maxi = feature.peak_local_max(cp.asnumpy(distance), labels=cp.asnumpy(image), footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = cp.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(np.array(local_maxi).T)] = True
    markers = nd_label(cp.asnumpy(local_maxi_mask))[0]
    segmented_image = segmentation.watershed(-cp.asnumpy(distance), markers, mask=cp.asnumpy(image))
    return cp.asnumpy(segmented_image), np.max(segmented_image)

# Step 7: Morphological Operations
def apply_morphological_operations(image, kernel_size=5):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    image = image.astype(np.uint8)
    closed = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)
    return closed, opened

# Full preprocessing pipeline
# Full preprocessing pipeline
def preprocess_pipeline(image):
    try:
        # Convert to grayscale
        grayscale_image = rgb2gray(image)

        # Apply preprocessing steps
        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        skull_stripped_image = skull_stripping(adf_filtered_image)
        tophat_image = top_hat_filtering(skull_stripped_image)
        contrasted_image = contrast_enhancement(tophat_image)
        binary_image, _ = get_binary(contrasted_image, thresh=230)
        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        closed_image_post, opened_image_post = apply_morphological_operations(segmented_image)

        # Resize the opened image for display
        resized_opened_image = transform.resize(opened_image_post, (128, 128))
        resized_opened_image = (resized_opened_image * 255).astype(np.uint8)  # Ensure uint8 format

        # Flatten and scale the image for the model
        flattened_image = resized_opened_image.flatten().reshape(1, -1)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_opened_image, contrasted_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise


# Classification Function
def classify_image(image, model_name):
    try:
        # Preprocess image
        scaled_features, opened_image, contrast_image = preprocess_pipeline(image)

        # Ensure opened_image is in uint8 for Gradio
        opened_image = (opened_image * 255).astype(np.uint8)

        # Predict using the selected model
        model = loaded_models[model_name]
        prediction = model.predict(scaled_features)

        return opened_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"


# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(
                    choices=list(models.keys()), label="Select Model", value="Random Forest"
                )
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Preprocessing
        preprocess_button.click(
            lambda image: preprocess_pipeline(image)[1], inputs=image_input, outputs=preprocess_output
        )

        # Classification
        classify_button.click(
            classify_image,
            inputs=[image_input, model_dropdown],
            outputs=[preprocess_output, prediction_output],
        )

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


In [24]:
import gradio as gr
import numpy as np
import cupy as cp
import cv2
from skimage import morphology, filters, segmentation, exposure, transform, feature
from skimage.color import rgb2gray
from scipy.ndimage import distance_transform_edt, label as nd_label
from sklearn.preprocessing import MinMaxScaler
from joblib import load

# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# Step 1: Anisotropic Diffusion Filter
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = cp.asarray(image.copy())  # Move image to GPU
    for _ in range(iterations):
        gradient_magnitude = cp.gradient(image)
        c = 1.0 / (1.0 + (gradient_magnitude[0] / beta) ** 2)
        diff_north = cp.roll(image, shift=1, axis=0) - image
        diff_south = cp.roll(image, shift=-1, axis=0) - image
        diff_east = cp.roll(image, shift=1, axis=1) - image
        diff_west = cp.roll(image, shift=-1, axis=1) - image
        image += alpha * (c * (diff_north + diff_south + diff_east + diff_west))
    return cp.asnumpy(image)  # Move back to CPU

# Step 2: Skull Stripping
def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=1500):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

# Step 3: Top-Hat Filtering
def top_hat_filtering(image, kernel_size=30):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    return cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)

# Step 4: Contrast Enhancement
def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    return ((contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image)) * 255).astype(np.uint8)

# Step 5: Binary Segmentation
def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    return image > thresh, thresh

# Step 6: Watershed Segmentation
def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    image = cp.asarray(image)
    distance = cp.asarray(distance_transform_edt(cp.asnumpy(image)))
    local_maxi = feature.peak_local_max(cp.asnumpy(distance), labels=cp.asnumpy(image), footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = cp.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(np.array(local_maxi).T)] = True
    markers = nd_label(cp.asnumpy(local_maxi_mask))[0]
    segmented_image = segmentation.watershed(-cp.asnumpy(distance), markers, mask=cp.asnumpy(image))
    return cp.asnumpy(segmented_image), np.max(segmented_image)

# Step 7: Morphological Operations
def apply_morphological_operations(image, kernel_size=5):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    image = image.astype(np.uint8)
    closed = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    opened = cv2.morphologyEx(closed, cv2.MORPH_OPEN, kernel)
    return closed, opened

# Full preprocessing pipeline
def preprocess_pipeline(image):
    try:
        grayscale_image = rgb2gray(image)
        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        skull_stripped_image = skull_stripping(adf_filtered_image)
        tophat_image = top_hat_filtering(skull_stripped_image)
        contrasted_image = contrast_enhancement(tophat_image)
        binary_image, _ = get_binary(contrasted_image, thresh=230)
        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        closed_image_post, opened_image_post = apply_morphological_operations(segmented_image)

        # ✅ Resize to 256x256 (ensures 65536 features)
        resized_opened_image = transform.resize(opened_image_post, (256, 256))
        resized_opened_image = (resized_opened_image * 255).astype(np.uint8)

        # ✅ Flatten and scale
        flattened_image = resized_opened_image.flatten().reshape(1, -1)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_opened_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise

# Classification Function with Dynamic Feature Handling
def classify_image(image, model_name):
    try:
        scaled_features, opened_image = preprocess_pipeline(image)
        model = loaded_models[model_name]

        # ✅ Dynamically match feature size
        expected_size = model.n_features_in_
        if scaled_features.shape[1] != expected_size:
            print(f"Resizing from {scaled_features.shape[1]} to {expected_size}")
            scaled_features = np.resize(scaled_features, (1, expected_size))  

        prediction = model.predict(scaled_features)
        return opened_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"

# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                classify_button = gr.Button("Classify")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


In [25]:
import gradio as gr
import numpy as np
import cupy as cp
import cv2
from skimage import morphology, filters, segmentation, exposure, transform, feature
from skimage.color import rgb2gray
from scipy.ndimage import distance_transform_edt, label as nd_label
from sklearn.preprocessing import MinMaxScaler
from joblib import load

# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}


# Step 1: Anisotropic Diffusion Filter (ADF)
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = img_as_float(image.copy())
    for _ in range(iterations):
        gradient_magnitude = gaussian_gradient_magnitude(image, sigma=1)
        c = 1.0 / (1.0 + (gradient_magnitude / beta) ** 2)
        diff_north = convolve(image, np.array([[0, 1, 0], [0, -1, 0], [0, 0, 0]]))
        diff_south = convolve(image, np.array([[0, 0, 0], [0, -1, 0], [0, 1, 0]]))
        diff_east = convolve(image, np.array([[0, 0, 0], [0, -1, 1], [0, 0, 0]]))
        diff_west = convolve(image, np.array([[0, 0, 0], [1, -1, 0], [0, 0, 0]]))
        image += alpha * (c * diff_north + c * diff_south + c * diff_east + c * diff_west)
    return img_as_ubyte(image)

# Step 2: Skull Stripping
def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=2000):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

# Step 3: Top-Hat Filtering
def top_hat_filtering(image, disk_size=30):
    footprint = morphology.disk(disk_size)
    return morphology.white_tophat(image, footprint)

# Step 4: Contrast Enhancement
def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    contrasted_image = (contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image))
    return img_as_ubyte(contrasted_image)

# Step 5: Binary Segmentation
def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    binary_image = image > thresh
    return binary_image, thresh

# Step 6: Watershed Segmentation
def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    distance = distance_transform_edt(image)
    local_maxi = feature.peak_local_max(distance, labels=image, footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = np.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(local_maxi.T)] = True
    markers = nd_label(local_maxi_mask)[0]  # Use `nd_label` here
    segmented_image = segmentation.watershed(-distance, markers, mask=image)
    nbr_labels = np.max(segmented_image)
    return segmented_image, nbr_labels


# Step 7: Morphological Operations
def apply_morphological_operations(segmented_image):
    if not isinstance(segmented_image, np.ndarray) or segmented_image.ndim != 2:
        return None, None  # Prevent errors if input is invalid
    
    closed_image = morphology.closing(segmented_image, morphology.disk(6))
    opened_image = morphology.opening(closed_image, morphology.disk(5))
    
    return closed_image, opened_image


# Full preprocessing pipeline
def preprocess_pipeline(image):
    try:
        grayscale_image = rgb2gray(image)
        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        skull_stripped_image = skull_stripping(adf_filtered_image)
        tophat_image = top_hat_filtering(skull_stripped_image)
        contrasted_image = contrast_enhancement(tophat_image)
        binary_image, _ = get_binary(contrasted_image, thresh=230)
        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        closed_image_post, opened_image_post = apply_morphological_operations(segmented_image)

        # ✅ Resize to 256x256 (ensures 65536 features)
        resized_opened_image = transform.resize(opened_image_post, (256, 256))
        resized_opened_image = (resized_opened_image * 255).astype(np.uint8)

        # ✅ Flatten and scale
        flattened_image = resized_opened_image.flatten().reshape(1, -1)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_opened_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise

# Classification Function with Dynamic Feature Handling
def classify_image(image, model_name):
    try:
        scaled_features, opened_image = preprocess_pipeline(image)
        model = loaded_models[model_name]

        # ✅ Dynamically match feature size
        expected_size = model.n_features_in_
        if scaled_features.shape[1] != expected_size:
            print(f"Resizing from {scaled_features.shape[1]} to {expected_size}")
            scaled_features = np.resize(scaled_features, (1, expected_size))  

        prediction = model.predict(scaled_features)
        return opened_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"

# ✅ Function for Preprocessing Button
def preprocess_image(image):
    _, processed_image = preprocess_pipeline(image)
    return processed_image

# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")  # ✅ Added Preprocessing Button
                preprocess_output = gr.Image(label="Preprocessed Image Preview")  # ✅ Preprocessing Output
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # ✅ Preprocessing Button Click
        preprocess_button.click(preprocess_image, inputs=image_input, outputs=preprocess_output)

        # ✅ Classification Button Click
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


In [3]:
import gradio as gr
import numpy as np
import cupy as cp
import cv2
from skimage.util import img_as_ubyte, img_as_float
from skimage import morphology, filters, segmentation, exposure, feature
from scipy.ndimage import convolve, gaussian_gradient_magnitude, distance_transform_edt
from scipy.ndimage import label as nd_label
from skimage.color import rgb2gray
from sklearn.preprocessing import MinMaxScaler
from joblib import load
from skimage import transform

# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# Step 1: Anisotropic Diffusion Filter (ADF)
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = img_as_float(image.copy())
    for _ in range(iterations):
        gradient_magnitude = gaussian_gradient_magnitude(image, sigma=1)
        c = 1.0 / (1.0 + (gradient_magnitude / beta) ** 2)
        diff_north = convolve(image, np.array([[0, 1, 0], [0, -1, 0], [0, 0, 0]]))
        diff_south = convolve(image, np.array([[0, 0, 0], [0, -1, 0], [0, 1, 0]]))
        diff_east = convolve(image, np.array([[0, 0, 0], [0, -1, 1], [0, 0, 0]]))
        diff_west = convolve(image, np.array([[0, 0, 0], [1, -1, 0], [0, 0, 0]]))
        image += alpha * (c * diff_north + c * diff_south + c * diff_east + c * diff_west)
    return img_as_ubyte(image)

# Step 2: Skull Stripping
def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=2000):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

# Step 3: Top-Hat Filtering
def top_hat_filtering(image, disk_size=30):
    footprint = morphology.disk(disk_size)
    return morphology.white_tophat(image, footprint)

# Step 4: Contrast Enhancement
def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    contrasted_image = (contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image))
    return img_as_ubyte(contrasted_image)

# Step 5: Binary Segmentation
def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    binary_image = image > thresh
    return binary_image, thresh

# Step 6: Watershed Segmentation
def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    distance = distance_transform_edt(image)
    local_maxi = feature.peak_local_max(distance, labels=image, footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = np.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(local_maxi.T)] = True
    markers = nd_label(local_maxi_mask)[0]
    segmented_image = segmentation.watershed(-distance, markers, mask=image)
    nbr_labels = np.max(segmented_image)
    return segmented_image, nbr_labels

# Step 7: Morphological Operations
def apply_morphological_operations(segmented_image):
    if not isinstance(segmented_image, np.ndarray) or segmented_image.ndim != 2:
        return None, None
    closed_image = morphology.closing(segmented_image, morphology.disk(6))
    opened_image = morphology.opening(closed_image, morphology.disk(5))
    return closed_image, opened_image

# Full preprocessing pipeline
def preprocess_pipeline(image):
    try:
        grayscale_image = rgb2gray(image)
        print(f"Grayscale Image Min: {grayscale_image.min()}, Max: {grayscale_image.max()}")

        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        print(f"ADF Filtered Min: {adf_filtered_image.min()}, Max: {adf_filtered_image.max()}")

        skull_stripped_image = skull_stripping(adf_filtered_image)
        print(f"Skull Stripped Min: {skull_stripped_image.min()}, Max: {skull_stripped_image.max()}")

        tophat_image = top_hat_filtering(skull_stripped_image)
        print(f"Tophat Filtered Min: {tophat_image.min()}, Max: {tophat_image.max()}")

        contrasted_image = contrast_enhancement(tophat_image)
        print(f"Contrast Enhanced Min: {contrasted_image.min()}, Max: {contrasted_image.max()}")

        binary_image, _ = get_binary(contrasted_image, thresh=230)
        print(f"Binary Image Min: {binary_image.min()}, Max: {binary_image.max()}")

        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        print(f"Segmented Image Min: {segmented_image.min()}, Max: {segmented_image.max()}")

        closed_image_post, opened_image_post = apply_morphological_operations(segmented_image)
        print(f"Opened Image Post Min: {opened_image_post.min()}, Max: {opened_image_post.max()}")

        # Resize to 256x256
        resized_opened_image = transform.resize(opened_image_post, (256, 256))
        resized_opened_image = (resized_opened_image * 255).astype(np.uint8)
        print(f"Resized Opened Image Min: {resized_opened_image.min()}, Max: {resized_opened_image.max()}")

        # Flatten and scale
        flattened_image = resized_opened_image.flatten().reshape(1, -1)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_opened_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise


# Classification Function
def classify_image(image, model_name):
    try:
        scaled_features, opened_image = preprocess_pipeline(image)
        model = loaded_models[model_name]

        # Dynamically match feature size
        expected_size = model.n_features_in_
        if scaled_features.shape[1] != expected_size:
            print(f"Resizing from {scaled_features.shape[1]} to {expected_size}")
            scaled_features = np.resize(scaled_features, (1, expected_size))

        prediction = model.predict(scaled_features)
        return opened_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"

# Preprocessing Button Function
def preprocess_image(image):
    _, processed_image = preprocess_pipeline(image)
    return processed_image

# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        preprocess_button.click(preprocess_image, inputs=image_input, outputs=preprocess_output)
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


C:\Users\Marwan Shamsan\AppData\Local\Temp\ipykernel_5684\2946655941.py:27: RuntimeWarning: invalid value encountered in divide
  reshaped_array = (reshaped_array - reshaped_array.min()) / (reshaped_array.max() - reshaped_array.min())  # Normalize
C:\Users\Marwan Shamsan\AppData\Local\Temp\ipykernel_5684\2946655941.py:28: RuntimeWarning: invalid value encountered in cast
  reshaped_array = (reshaped_array * 255).astype(np.uint8)  # Convert to 8-bit image
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\Marwan Shamsan\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\Marwan Shamsan\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forci

Grayscale Image Min: 0.0, Max: 1.0
ADF Filtered Min: 0, Max: 255
Skull Stripped Min: 0, Max: 255
Tophat Filtered Min: 0, Max: 255
Contrast Enhanced Min: 0, Max: 255
Binary Image Min: False, Max: True
Segmented Image Min: 0, Max: 2
Opened Image Post Min: 0, Max: 0
Resized Opened Image Min: 0, Max: 0


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

# Define the missing class before loading the pickle file
class ImagePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # No training needed

    def transform(self, X):
        processed_images = []
        for image in X:
            try:
                # Apply the full pipeline to each image
                adf_filtered_image = anisotropic_diffusion_filter(image)
                skull_stripped_image = skull_stripping(adf_filtered_image)
                tophat_image = top_hat_filtering(skull_stripped_image)
                contrasted_image = contrast_enhancement(tophat_image)
                binary_image, _ = get_binary(contrasted_image, thresh=230)
                segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
                _, opened_image = apply_morphological_operations(segmented_image)

                # Resize to 256x256
                resized_opened_image = transform.resize(opened_image, (256, 256), anti_aliasing=True)
                if resized_opened_image.max() > 0:
                    resized_opened_image = (resized_opened_image / resized_opened_image.max()) * 255
                resized_opened_image = resized_opened_image.astype(np.uint8)

                # Flatten the image
                flattened_image = resized_opened_image.flatten().reshape(1, -1)
                
                processed_images.append(flattened_image)
            except Exception as e:
                print(f"Error processing image: {e}")
                processed_images.append(None)

        return np.vstack(processed_images)  # Stack all images together

# ✅ Now Load the Preprocessing Pipeline
import joblib
preprocessor = joblib.load(r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl")
print("Preprocessing pipeline loaded successfully!")


Preprocessing pipeline loaded successfully!


In [28]:
import gradio as gr
import numpy as np
import joblib
from joblib import load
from skimage.color import rgb2gray

# Load preprocessing pipeline
preprocessing_pipeline_path = r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl"
preprocessor = joblib.load(preprocessing_pipeline_path)

# Load trained models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# ✅ Function to preprocess image using the saved pipeline
def preprocess_image(image):
    try:
        grayscale_image = rgb2gray(image)  # Convert to grayscale if needed
        processed_features = preprocessor.transform([grayscale_image])  # Apply the saved pipeline
        return processed_features.reshape(256, 256)  # Return as an image for preview
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return None

# ✅ Classification Function Using Preprocessed Image
def classify_image(image, model_name):
    try:
        processed_features = preprocess_image(image)
        if processed_features is None:
            return None, "Error in preprocessing"

        model = loaded_models[model_name]

        # Ensure feature size matches model expectations
        expected_size = model.n_features_in_
        processed_features = processed_features.flatten().reshape(1, -1)  # Flatten before resizing
        if processed_features.shape[1] != expected_size:
            processed_features = np.resize(processed_features, (1, expected_size))

        prediction = model.predict(processed_features)
        return image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"

# ✅ Gradio GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        preprocess_button.click(preprocess_image, inputs=image_input, outputs=preprocess_output)
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


In [29]:
import gradio as gr
import numpy as np
import joblib
from joblib import load
from skimage.color import rgb2gray

# Load preprocessing pipeline
preprocessing_pipeline_path = r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl"
preprocessor = joblib.load(preprocessing_pipeline_path)

# Load trained models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# ✅ Function to preprocess image using the saved pipeline
def preprocess_image(image):
    try:
        grayscale_image = rgb2gray(image)  # Convert to grayscale if needed
        processed_features = preprocessor.transform([grayscale_image])  # Apply the saved pipeline
        preprocessed_image = processed_features.reshape(256, 256)  # Reshape for image preview
        return image, preprocessed_image
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return image, None

# ✅ Classification Function Using Preprocessed Image
def classify_image(image, model_name):
    try:
        grayscale_image = rgb2gray(image)  # Ensure grayscale conversion
        processed_features = preprocessor.transform([grayscale_image])  # Apply preprocessing

        if processed_features is None:
            return image, None, "Error in preprocessing"

        model = loaded_models[model_name]

        # Ensure feature size matches model expectations
        expected_size = model.n_features_in_
        processed_features = processed_features.flatten().reshape(1, -1)  # Flatten before resizing
        if processed_features.shape[1] != expected_size:
            processed_features = np.resize(processed_features, (1, expected_size))

        prediction = model.predict(processed_features)
        preprocessed_image = processed_features.reshape(256, 256)  # Preprocessed image for display
        return image, preprocessed_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return image, None, "Error"

# ✅ Gradio GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy", interactive=True)
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Button actions
        preprocess_button.click(preprocess_image, inputs=image_input, outputs=[image_input, preprocess_output])
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[image_input, preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7890

To create a public link, set `share=True` in `launch()`.


In [30]:
import gradio as gr
import numpy as np
import joblib
from joblib import load
from skimage.color import rgb2gray

# Load preprocessing pipeline
preprocessing_pipeline_path = r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl"
preprocessor = joblib.load(preprocessing_pipeline_path)

# Load trained models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# ✅ Function to dynamically reshape a flattened array
def dynamic_reshape(flattened_array):
    """Dynamically reshapes a flattened array into a square or near-square shape."""
    length = flattened_array.shape[0]
    side = int(np.sqrt(length))  # Find the largest square shape possible
    reshaped_array = flattened_array[:side * side].reshape(side, side)  # Crop excess and reshape
    return reshaped_array

# ✅ Function to preprocess image using the saved pipeline
def preprocess_image(image):
    try:
        grayscale_image = rgb2gray(image)  # Convert to grayscale if needed
        processed_features = preprocessor.transform([grayscale_image])  # Apply the saved pipeline
        preprocessed_image = dynamic_reshape(processed_features.flatten())  # Dynamically reshape
        return image, preprocessed_image
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return image, None

# ✅ Classification Function Using Preprocessed Image
def classify_image(image, model_name):
    try:
        grayscale_image = rgb2gray(image)  # Ensure grayscale conversion
        processed_features = preprocessor.transform([grayscale_image])  # Apply preprocessing

        if processed_features is None:
            return image, None, "Error in preprocessing"

        model = loaded_models[model_name]

        # Ensure feature size matches model expectations
        expected_size = model.n_features_in_
        processed_features = processed_features.flatten().reshape(1, -1)  # Flatten before resizing
        if processed_features.shape[1] != expected_size:
            processed_features = np.resize(processed_features, (1, expected_size))

        prediction = model.predict(processed_features)
        preprocessed_image = dynamic_reshape(processed_features.flatten())  # Dynamically reshape
        return image, preprocessed_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return image, None, "Error"

# ✅ Gradio GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy", interactive=True)
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Button actions
        preprocess_button.click(preprocess_image, inputs=image_input, outputs=[image_input, preprocess_output])
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[image_input, preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


In [31]:
import gradio as gr
import numpy as np
import joblib
from joblib import load
from skimage.color import rgb2gray

# Load preprocessing pipeline
preprocessing_pipeline_path = r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl"
preprocessor = joblib.load(preprocessing_pipeline_path)

# Load trained models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# ✅ Function to dynamically resize an image for visualization
def resize_for_visualization(flattened_array, target_size=(256, 256)):
    """Resizes the flattened array to the target size for visualization."""
    reshaped_array = flattened_array[: np.prod(target_size)].reshape(target_size)  # Crop and reshape
    reshaped_array = (reshaped_array - reshaped_array.min()) / (reshaped_array.max() - reshaped_array.min())  # Normalize
    reshaped_array = (reshaped_array * 255).astype(np.uint8)  # Convert to 8-bit image
    return reshaped_array

# ✅ Function to preprocess image using the saved pipeline
def preprocess_image(image):
    try:
        grayscale_image = rgb2gray(image)  # Convert to grayscale if needed
        processed_features = preprocessor.transform([grayscale_image])  # Apply the saved pipeline
        visualization_image = resize_for_visualization(processed_features.flatten())  # Resize for preview
        return image, visualization_image
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return image, None

# ✅ Classification Function Using Preprocessed Image
def classify_image(image, model_name):
    try:
        grayscale_image = rgb2gray(image)  # Ensure grayscale conversion
        processed_features = preprocessor.transform([grayscale_image])  # Apply preprocessing

        if processed_features is None:
            return image, None, "Error in preprocessing"

        model = loaded_models[model_name]

        # Ensure feature size matches model expectations
        expected_size = model.n_features_in_
        flattened_features = processed_features.flatten().reshape(1, -1)  # Flatten before resizing
        if flattened_features.shape[1] != expected_size:
            flattened_features = np.resize(flattened_features, (1, expected_size))

        prediction = model.predict(flattened_features)
        visualization_image = resize_for_visualization(processed_features.flatten())  # Resize for preview
        return image, visualization_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return image, None, "Error"

# ✅ Gradio GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy", interactive=True)
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Button actions
        preprocess_button.click(preprocess_image, inputs=image_input, outputs=[image_input, preprocess_output])
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[image_input, preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7892

To create a public link, set `share=True` in `launch()`.


In [1]:
import gradio as gr
import numpy as np
import cupy as cp
import cv2
from skimage.util import img_as_ubyte, img_as_float
from skimage import morphology, filters, segmentation, exposure, feature
from scipy.ndimage import convolve, gaussian_gradient_magnitude, distance_transform_edt
from scipy.ndimage import label as nd_label
from skimage.color import rgb2gray
from sklearn.preprocessing import MinMaxScaler
from joblib import load
from skimage import transform

# Load models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# Step 1: Anisotropic Diffusion Filter (ADF)
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = img_as_float(image.copy())
    for _ in range(iterations):
        gradient_magnitude = gaussian_gradient_magnitude(image, sigma=1)
        c = 1.0 / (1.0 + (gradient_magnitude / beta) ** 2)
        diff_north = convolve(image, np.array([[0, 1, 0], [0, -1, 0], [0, 0, 0]]))
        diff_south = convolve(image, np.array([[0, 0, 0], [0, -1, 0], [0, 1, 0]]))
        diff_east = convolve(image, np.array([[0, 0, 0], [0, -1, 1], [0, 0, 0]]))
        diff_west = convolve(image, np.array([[0, 0, 0], [1, -1, 0], [0, 0, 0]]))
        image += alpha * (c * diff_north + c * diff_south + c * diff_east + c * diff_west)
    return img_as_ubyte(image)

# Step 2: Skull Stripping
def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=2000):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

# Step 3: Top-Hat Filtering
def top_hat_filtering(image, disk_size=30):
    footprint = morphology.disk(disk_size)
    return morphology.white_tophat(image, footprint)

# Step 4: Contrast Enhancement
def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    contrasted_image = (contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image))
    return img_as_ubyte(contrasted_image)

# Step 5: Binary Segmentation
def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    binary_image = image > thresh
    return binary_image, thresh

# Step 6: Watershed Segmentation
def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    distance = distance_transform_edt(image)
    local_maxi = feature.peak_local_max(distance, labels=image, footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = np.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(local_maxi.T)] = True
    markers = nd_label(local_maxi_mask)[0]
    segmented_image = segmentation.watershed(-distance, markers, mask=image)
    nbr_labels = np.max(segmented_image)
    return segmented_image, nbr_labels

# Step 7: Morphological Operations
def apply_morphological_operations(segmented_image):
    if not isinstance(segmented_image, np.ndarray) or segmented_image.ndim != 2:
        return None, None
    closed_image = morphology.closing(segmented_image, morphology.disk(6))
    opened_image = morphology.opening(closed_image, morphology.disk(5))
    return closed_image, opened_image

# Full preprocessing pipeline
def preprocess_pipeline(image):
    try:
        grayscale_image = rgb2gray(image)
        print(f"Grayscale Image Min: {grayscale_image.min()}, Max: {grayscale_image.max()}")

        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        print(f"ADF Filtered Min: {adf_filtered_image.min()}, Max: {adf_filtered_image.max()}")

        skull_stripped_image = skull_stripping(adf_filtered_image)
        print(f"Skull Stripped Min: {skull_stripped_image.min()}, Max: {skull_stripped_image.max()}")

        tophat_image = top_hat_filtering(skull_stripped_image)
        print(f"Tophat Filtered Min: {tophat_image.min()}, Max: {tophat_image.max()}")

        contrasted_image = contrast_enhancement(tophat_image)
        print(f"Contrast Enhanced Min: {contrasted_image.min()}, Max: {contrasted_image.max()}")

        binary_image, _ = get_binary(contrasted_image, thresh=230)
        print(f"Binary Image Min: {binary_image.min()}, Max: {binary_image.max()}")

        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        print(f"Segmented Image Min: {segmented_image.min()}, Max: {segmented_image.max()}")

        closed_image_post, opened_image_post = apply_morphological_operations(segmented_image)
        print(f"Opened Image Post Min: {opened_image_post.min()}, Max: {opened_image_post.max()}")

        # Resize to 256x256
        resized_opened_image = transform.resize(opened_image_post, (256, 256))
        resized_opened_image = (resized_opened_image * 255).astype(np.uint8)
        print(f"Resized Opened Image Min: {resized_opened_image.min()}, Max: {resized_opened_image.max()}")

        # Flatten and scale
        flattened_image = resized_opened_image.flatten().reshape(1, -1)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_opened_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise


# Classification Function
def classify_image(image, model_name):
    try:
        scaled_features, opened_image = preprocess_pipeline(image)
        model = loaded_models[model_name]

        # Dynamically match feature size
        expected_size = model.n_features_in_
        if scaled_features.shape[1] != expected_size:
            print(f"Resizing from {scaled_features.shape[1]} to {expected_size}")
            scaled_features = np.resize(scaled_features, (1, expected_size))

        prediction = model.predict(scaled_features)
        return opened_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"

# Preprocessing Button Function
def preprocess_image(image):
    _, processed_image = preprocess_pipeline(image)
    return processed_image

# GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy")
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        preprocess_button.click(preprocess_image, inputs=image_input, outputs=preprocess_output)
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [1]:
from sklearn.base import BaseEstimator, TransformerMixin

# Define the missing class before loading the pickle file
class ImagePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self  # No training needed

    def transform(self, X):
        processed_images = []
        for image in X:
            try:
                # Apply the full pipeline to each image
                adf_filtered_image = anisotropic_diffusion_filter(image)
                skull_stripped_image = skull_stripping(adf_filtered_image)
                tophat_image = top_hat_filtering(skull_stripped_image)
                contrasted_image = contrast_enhancement(tophat_image)
                binary_image, _ = get_binary(contrasted_image, thresh=230)
                segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
                _, opened_image = apply_morphological_operations(segmented_image)

                # Resize to 256x256
                resized_opened_image = transform.resize(opened_image, (256, 256), anti_aliasing=True)
                if resized_opened_image.max() > 0:
                    resized_opened_image = (resized_opened_image / resized_opened_image.max()) * 255
                resized_opened_image = resized_opened_image.astype(np.uint8)

                # Flatten the image
                flattened_image = resized_opened_image.flatten().reshape(1, -1)
                
                processed_images.append(flattened_image)
            except Exception as e:
                print(f"Error processing image: {e}")
                processed_images.append(None)

        return np.vstack(processed_images)  # Stack all images together

# ✅ Now Load the Preprocessing Pipeline
import joblib
preprocessor = joblib.load(r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl")
print("Preprocessing pipeline loaded successfully!")


Preprocessing pipeline loaded successfully!


In [2]:
import gradio as gr
import numpy as np
import joblib
from joblib import load
from skimage.color import rgb2gray

# Load preprocessing pipeline
preprocessing_pipeline_path = r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl"
preprocessor = joblib.load(preprocessing_pipeline_path)

# Load trained models
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}

loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# ✅ Function to dynamically resize an image for visualization
def resize_for_visualization(flattened_array, target_size=(256, 256)):
    """Resizes the flattened array to the target size for visualization."""
    reshaped_array = flattened_array[: np.prod(target_size)].reshape(target_size)  # Crop and reshape
    reshaped_array = (reshaped_array - reshaped_array.min()) / (reshaped_array.max() - reshaped_array.min())  # Normalize
    reshaped_array = (reshaped_array * 255).astype(np.uint8)  # Convert to 8-bit image
    return reshaped_array

# ✅ Function to preprocess image using the saved pipeline
def preprocess_image(image):
    try:
        grayscale_image = rgb2gray(image)  # Convert to grayscale if needed
        processed_features = preprocessor.transform([grayscale_image])  # Apply the saved pipeline
        visualization_image = resize_for_visualization(processed_features.flatten())  # Resize for preview
        return image, visualization_image
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return image, None

# Define label mapping for predictions
label_mapping = {
    0: "Tumor Detected",         # Class 0 -> Healthy
    1: "Healthy",  # Class 1 -> Tumor Detected2: "Cancer Detected"  # (Optional) Example for a third class
}

# ✅ Classification Function Using Preprocessed Image
def classify_image(image, model_name):
    try:
        grayscale_image = rgb2gray(image)  # Ensure grayscale conversion
        processed_features = preprocessor.transform([grayscale_image])  # Apply preprocessing

        if processed_features is None:
            return image, None, "Error in preprocessing"

        model = loaded_models[model_name]

        # Ensure feature size matches model expectations
        expected_size = model.n_features_in_
        flattened_features = processed_features.flatten().reshape(1, -1)  # Flatten before resizing
        if flattened_features.shape[1] != expected_size:
            flattened_features = np.resize(flattened_features, (1, expected_size))

        # Get the numerical prediction
        prediction = model.predict(flattened_features)[0]  # Single prediction

        # Map the numerical label to a meaningful description
        label = label_mapping.get(prediction, "Unknown")  # Fallback to "Unknown" if label is missing

        # Resize for visualization
        visualization_image = resize_for_visualization(processed_features.flatten())  # Resize for preview
        return image, visualization_image, f"Result: {label}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return image, None, "Error"


# ✅ Gradio GUI
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")

        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy", interactive=True)
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)

        # Button actions
        preprocess_button.click(preprocess_image, inputs=image_input, outputs=[image_input, preprocess_output])
        classify_button.click(classify_image, inputs=[image_input, model_dropdown], outputs=[image_input, preprocess_output, prediction_output])

    demo.launch()

if __name__ == "__main__":
    gui()


AttributeError: Can't get attribute 'ImagePreprocessor' on <module '__main__'>

In [3]:
# Cell 1: Imports, model loading, and pipeline functions

import gradio as gr
import numpy as np
import cupy as cp
import cv2
from skimage.util import img_as_ubyte, img_as_float
from skimage import morphology, filters, segmentation, exposure, feature, transform
from scipy.ndimage import convolve, gaussian_gradient_magnitude, distance_transform_edt
from scipy.ndimage import label as nd_label
from skimage.color import rgb2gray
from sklearn.preprocessing import MinMaxScaler
from joblib import load

# ---------------------------
# Load trained models
# ---------------------------
models = {
    "Random Forest": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_random_forest_model.pkl",
    "KNN": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\best_KNN_model.pkl",
    "Decision Tree": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\Decision_tree_model.pkl",
    "Gaussian NB": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\GaussianNB_model.pkl",
    "Logistic Regression": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\logistic_regression_model.pkl",
    "SVM": r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\svm_model.pkl",
}
loaded_models = {model_name: load(model_path) for model_name, model_path in models.items()}

# ---------------------------
# Preprocessing Pipeline Functions
# ---------------------------
def anisotropic_diffusion_filter(image, alpha=0.1, beta=0.1, iterations=3):
    image = img_as_float(image.copy())
    for _ in range(iterations):
        gradient_magnitude = gaussian_gradient_magnitude(image, sigma=1)
        c = 1.0 / (1.0 + (gradient_magnitude / beta) ** 2)
        diff_north = convolve(image, np.array([[0, 1, 0], [0, -1, 0], [0, 0, 0]]))
        diff_south = convolve(image, np.array([[0, 0, 0], [0, -1, 0], [0, 1, 0]]))
        diff_east = convolve(image, np.array([[0, 0, 0], [0, -1, 1], [0, 0, 0]]))
        diff_west = convolve(image, np.array([[0, 0, 0], [1, -1, 0], [0, 0, 0]]))
        image += alpha * (c * diff_north + c * diff_south + c * diff_east + c * diff_west)
    return img_as_ubyte(image)

def skull_stripping(image, se_closing=None, se_erosion=None, skull_remove_area=2000):
    if se_closing is None:
        se_closing = morphology.disk(15)
    if se_erosion is None:
        se_erosion = morphology.disk(30)
    threshold_value = filters.threshold_otsu(image)
    binary_image = image > threshold_value
    filled_image = morphology.closing(binary_image, se_closing)
    filled_image = morphology.remove_small_holes(filled_image, area_threshold=skull_remove_area)
    eroded_image = morphology.erosion(filled_image, se_erosion)
    return np.where(eroded_image, image, 0)

def top_hat_filtering(image, disk_size=30):
    footprint = morphology.disk(disk_size)
    return morphology.white_tophat(image, footprint)

def contrast_enhancement(image):
    contrasted_image = exposure.equalize_hist(image)
    contrasted_image = (contrasted_image - np.min(contrasted_image)) / (np.max(contrasted_image) - np.min(contrasted_image))
    return img_as_ubyte(contrasted_image)

def get_binary(image, thresh=None):
    if thresh is None:
        thresh = filters.threshold_otsu(image)
    binary_image = image > thresh
    return binary_image, thresh

def watershed_segmentation(image, se_peak_local=np.ones((3, 3))):
    distance = distance_transform_edt(image)
    local_maxi = feature.peak_local_max(distance, labels=image, footprint=se_peak_local, exclude_border=False)
    local_maxi_mask = np.zeros_like(image, dtype=bool)
    local_maxi_mask[tuple(local_maxi.T)] = True
    markers = nd_label(local_maxi_mask)[0]
    segmented_image = segmentation.watershed(-distance, markers, mask=image)
    nbr_labels = np.max(segmented_image)
    return segmented_image, nbr_labels

def apply_morphological_operations(segmented_image):
    if not isinstance(segmented_image, np.ndarray) or segmented_image.ndim != 2:
        return None, None
    closed_image = morphology.closing(segmented_image, morphology.disk(6))
    opened_image = morphology.opening(closed_image, morphology.disk(5))
    return closed_image, opened_image

def preprocess_pipeline(image):
    try:
        # Convert to grayscale
        grayscale_image = rgb2gray(image)
        print(f"Grayscale Image Min: {grayscale_image.min()}, Max: {grayscale_image.max()}")

        adf_filtered_image = anisotropic_diffusion_filter(grayscale_image)
        print(f"ADF Filtered Min: {adf_filtered_image.min()}, Max: {adf_filtered_image.max()}")

        skull_stripped_image = skull_stripping(adf_filtered_image)
        print(f"Skull Stripped Min: {skull_stripped_image.min()}, Max: {skull_stripped_image.max()}")

        tophat_image = top_hat_filtering(skull_stripped_image)
        print(f"Tophat Filtered Min: {tophat_image.min()}, Max: {tophat_image.max()}")

        contrasted_image = contrast_enhancement(tophat_image)
        print(f"Contrast Enhanced Min: {contrasted_image.min()}, Max: {contrasted_image.max()}")

        # Here thresh is set to 230 (assumes an 8-bit image after contrast enhancement)
        binary_image, _ = get_binary(contrasted_image, thresh=230)
        print(f"Binary Image Min: {binary_image.min()}, Max: {binary_image.max()}")

        segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
        print(f"Segmented Image Min: {segmented_image.min()}, Max: {segmented_image.max()}")

        closed_image_post, opened_image_post = apply_morphological_operations(segmented_image)
        print(f"Opened Image Post Min: {opened_image_post.min()}, Max: {opened_image_post.max()}")

        # Resize to 256x256 for further processing/visualization
        resized_opened_image = transform.resize(opened_image_post, (256, 256), anti_aliasing=True)
        resized_opened_image = (resized_opened_image * 255).astype(np.uint8)
        print(f"Resized Opened Image Min: {resized_opened_image.min()}, Max: {resized_opened_image.max()}")

        # Flatten and scale features
        flattened_image = resized_opened_image.flatten().reshape(1, -1)
        from sklearn.preprocessing import MinMaxScaler  # (re-import if needed)
        scaler = MinMaxScaler()
        scaled_features = scaler.fit_transform(flattened_image)

        return scaled_features, resized_opened_image
    except Exception as e:
        print(f"Error in preprocessing pipeline: {e}")
        raise

# Additional helper functions for GUI (used later)
def preprocess_image(image):
    """Helper function for the GUI Preprocess button."""
    _, processed_image = preprocess_pipeline(image)
    return processed_image

def classify_image(image, model_name):
    try:
        scaled_features, opened_image = preprocess_pipeline(image)
        model = loaded_models[model_name]
        # Adjust feature size if necessary
        expected_size = model.n_features_in_
        if scaled_features.shape[1] != expected_size:
            print(f"Resizing from {scaled_features.shape[1]} to {expected_size}")
            scaled_features = np.resize(scaled_features, (1, expected_size))
        prediction = model.predict(scaled_features)
        return opened_image, f"Predicted Label: {prediction[0]}"
    except Exception as e:
        print(f"Error in classification: {e}")
        return None, "Error"

print("Cell 1 executed: All imports, models, and preprocessing functions are loaded.")


Cell 1 executed: All imports, models, and preprocessing functions are loaded.


In [4]:
# Cell 2: Define ImagePreprocessor class and load the preprocessing pipeline

from sklearn.base import BaseEstimator, TransformerMixin
import joblib  # Ensure joblib is imported

class ImagePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass  # No initialization parameters needed

    def fit(self, X, y=None):
        return self  # No fitting needed

    def transform(self, X):
        processed_images = []
        for image in X:
            try:
                # Apply the same pipeline as defined above
                adf_filtered_image = anisotropic_diffusion_filter(image)
                skull_stripped_image = skull_stripping(adf_filtered_image)
                tophat_image = top_hat_filtering(skull_stripped_image)
                contrasted_image = contrast_enhancement(tophat_image)
                binary_image, _ = get_binary(contrasted_image, thresh=230)
                segmented_image, _ = watershed_segmentation(binary_image, morphology.disk(10))
                _, opened_image = apply_morphological_operations(segmented_image)
                # Resize to 256x256 for consistency
                resized_opened_image = transform.resize(opened_image, (256, 256), anti_aliasing=True)
                if resized_opened_image.max() > 0:
                    resized_opened_image = (resized_opened_image / resized_opened_image.max()) * 255
                resized_opened_image = resized_opened_image.astype(np.uint8)
                flattened_image = resized_opened_image.flatten().reshape(1, -1)
                processed_images.append(flattened_image)
            except Exception as e:
                print(f"Error processing image: {e}")
                processed_images.append(None)
        return np.vstack(processed_images)

# ---------------------------
# Load the preprocessing pipeline from disk
# ---------------------------
preprocessing_pipeline_path = r"C:\Marwan APU\Sem-2\PR\brine\new_pkl\preprocessing_pipeline.pkl"
preprocessor = joblib.load(preprocessing_pipeline_path)
print("Cell 2 executed: Preprocessing pipeline loaded successfully!")


Cell 2 executed: Preprocessing pipeline loaded successfully!


In [5]:
# Cell 3: Gradio GUI Launch

import gradio as gr
import numpy as np
import joblib
from skimage.color import rgb2gray

# --- Helper function for visualization ---
def resize_for_visualization(flattened_array, target_size=(256, 256)):
    """Resizes a flattened array back to a 2D image for visualization."""
    reshaped_array = flattened_array[: np.prod(target_size)].reshape(target_size)
    # Normalize to [0, 255]
    reshaped_array = (reshaped_array - reshaped_array.min()) / (reshaped_array.max() - reshaped_array.min() + 1e-8)
    reshaped_array = (reshaped_array * 255).astype(np.uint8)
    return reshaped_array

# --- Redefine functions that use the preprocessor (if needed) ---
def preprocess_image_gui(image):
    """
    Uses the saved preprocessor to transform the image and then 
    resizes the output for visualization.
    """
    try:
        grayscale_image = rgb2gray(image)
        processed_features = preprocessor.transform([grayscale_image])
        visualization_image = resize_for_visualization(processed_features.flatten())
        return image, visualization_image
    except Exception as e:
        print(f"Error in preprocessing (GUI): {e}")
        return image, None

def classify_image_gui(image, model_name):
    """
    Applies the saved preprocessor and then uses the selected model for classification.
    Also resizes the processed features for preview.
    """
    try:
        grayscale_image = rgb2gray(image)
        processed_features = preprocessor.transform([grayscale_image])
        if processed_features is None:
            return image, None, "Error in preprocessing"
        model = loaded_models[model_name]
        # Resize features if needed
        expected_size = model.n_features_in_
        flattened_features = processed_features.flatten().reshape(1, -1)
        if flattened_features.shape[1] != expected_size:
            flattened_features = np.resize(flattened_features, (1, expected_size))
        prediction = model.predict(flattened_features)[0]
        # Map prediction to a label (adjust mapping as needed)
        label_mapping = {0: "Tumor Detected", 1: "Healthy"}
        label = label_mapping.get(prediction, "Unknown")
        visualization_image = resize_for_visualization(processed_features.flatten())
        return image, visualization_image, f"Result: {label}"
    except Exception as e:
        print(f"Error in classification (GUI): {e}")
        return image, None, "Error"

# --- Gradio Interface ---
def gui():
    with gr.Blocks() as demo:
        gr.Markdown("## Image Classification Demo")
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(label="Upload a Photo", type="numpy", interactive=True)
                preprocess_button = gr.Button("Preprocess")
                preprocess_output = gr.Image(label="Preprocessed Image Preview")
            with gr.Column():
                model_dropdown = gr.Dropdown(choices=list(models.keys()), label="Select Model", value="Random Forest")
                classify_button = gr.Button("Classify")
                prediction_output = gr.Textbox(label="Predicted Label", interactive=False)
        
        # When the Preprocess button is clicked:
        preprocess_button.click(
            preprocess_image_gui, 
            inputs=image_input, 
            outputs=[image_input, preprocess_output]
        )
        # When the Classify button is clicked:
        classify_button.click(
            classify_image_gui, 
            inputs=[image_input, model_dropdown], 
            outputs=[image_input, preprocess_output, prediction_output]
        )
    demo.launch()

gui()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\Marwan Shamsan\anaconda3\Lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\Marwan Shamsan\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
